In [13]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler


customers = pd.read_csv('Customers.csv')
transactions = pd.read_csv('Transactions.csv')
products = pd.read_csv('Products.csv')



In [14]:
# Merge data
data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')


# Rename price columns for clarity
data.rename(columns={'Price_x': 'TransactionPrice', 'Price_y': 'ProductPrice'}, inplace=True)

# Feature engineering
customer_features = data.groupby('CustomerID').agg({
    'TotalValue': 'sum',  # Total spending
    'TransactionID': 'count',  # Purchase frequency
    'TransactionPrice': 'mean',  # Average transaction value
    'Category': lambda x: x.mode()[0]  # Favorite product category
}).rename(columns={
    'TotalValue': 'TotalSpending',
    'TransactionID': 'PurchaseFrequency',
    'TransactionPrice': 'AvgTransactionValue',
    'Category': 'FavoriteCategory'
})

In [15]:

# Encode categorical feature
customer_features = pd.get_dummies(customer_features, columns=['FavoriteCategory'], drop_first=True)

# Scale the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features)

# Compute similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(scaled_features)

In [16]:
# Create Lookalike recommendations
lookalikes = {}
for i, customer_id in enumerate(customer_features.index):
    similar_indices = similarity_matrix[i].argsort()[::-1][1:4]  # Top 3 similar customers
    similar_customers = customer_features.index[similar_indices]
    similarity_scores = similarity_matrix[i][similar_indices]
    lookalikes[customer_id] = list(zip(similar_customers, similarity_scores))

# Prepare the Lookalike.csv file
lookalike_df = pd.DataFrame([
    {"CustomerID": cust, "Lookalikes": str(lookalikes[cust])}
    for cust in customer_features.index[:20]  # First 20 customers
])
lookalike_df.to_csv(r"C:\Users\saisa\Downloads\Bukkapatnam_SaiSashank_Lookalike.csv", index=False)
